In [30]:
import pickle
import pandas as pd

In [31]:
# Load in Eoin's saved sample data and models and scaler

df = pd.read_csv('sample_data.csv')

# scaler for transforming data
file = open("scaler.pkl",'rb')
scaler = pickle.load(file)
file.close()

# ML model
file = open("mlp.pkl",'rb')
mlp = pickle.load(file)
file.close()

# Screening model
file = open("screening_model.pkl",'rb')
screening_model = pickle.load(file)
file.close()

In [33]:
# Just indexing column not needed
del df['Unnamed: 0']

# Target growth rates to predict
target = df.growth_rate.values
del df['growth_rate']

In [34]:
# Transform data for model input
scaled_data = scaler.transform(df)

In [35]:
df.head()

,previous_gr,week,nextCover,previousCover,twoMonN,maxtp,mintp,rain,glorad,p_maxtp,p_mintp,p_rain,p_glorad
0,60.000000,39,6,5,86.9875,12.029022,7.550987,37.581201,568.060533,16.785681,8.815410,9.330664,868.223704
1,83.333333,17,7,3,0.0000,15.842857,6.700000,2.000000,1114.714286,13.900000,7.333333,12.500000,863.333333
2,45.454545,38,7,11,22.0211,15.958844,9.187494,46.715915,809.072031,13.086498,6.528831,28.843104,852.023553
3,90.000000,22,12,7,0.0000,13.110373,7.784655,43.160609,959.094450,15.912522,8.988780,8.316370,1423.048233
4,100.000000,19,8,6,0.0000,14.675000,8.850000,7.000000,1800.250000,12.716667,4.916667,18.500000,1400.333333


## It's probably easy to just define a function for the whole hybrid model

In [49]:
def hybrid_model(screening_model, mlp, most, x):
    """
    Input: Screening model, Machine learning model MLP, MoSt model, queries x
    Return: Prediction for datapoint x
    """
    
    final_predictions = list()
    
    screening_preds = screening_model.predict(scaled_data)
    mlp_preds = mlp.predict(x)
    mlp_preds[mlp_preds < 0] = 0
    most_preds = most.predict(x)
    
    for i in range(len(screening_preds)):
        model_to_use = screening_preds[i]
        
        if model_to_use == 1:
            final_predictions.append(mlp_preds[i])
            
        elif model_to_use == 0:
            final_predictions.append(most_preds[i])
            
    return final_predictions

In [50]:
hybrid_predictions = hybrid_model(screening_model, mlp, most, scaled_data)